In [18]:

import psycopg2
from psycopg2 import sql
import json
# 全局数据库连接参数
DB_CONFIG = {
    'host': 'bingviz-metrics-storage-dup.postgres.database.azure.com',
    'port': '5432',
    'dbname': 'metrics',
    'user': 'bingviz_metrics',
    'password': "xb;Rt&F4~]XWj:@r-7.df'QD"
}
class AgentPGSQL():
    def __init__(self):
        print('init_agentSQL')
        self.schema_name = 'sapphire'
        self.table_name = 'sapphire_engagement_metrics_master'
        self.conn = self.connect_to_postgres()
        self.cursor = self.conn.cursor()
        # 关闭游标和连接
    def __del__(self) -> None:
        print('del-运行')
        self.cursor.close()
        self.conn.close()
    def chat(self,sql_query:str,debug=False)-> str:
        self.conn = self.connect_to_postgres()
        self.cursor = self.conn.cursor()
        """
        Decode the output of GPT
        """
        err_tb_names=['your_table_name',r"{}.{}","table_name"]
        for i  in err_tb_names:
            if i in sql_query:
                sql_query=sql_query.replace(i,"your_schema.your_table")
        sql_query=sql_query.replace("your_schema.your_table", f"{self.schema_name}.{self.table_name}")
        
        if debug==False:
            sql_query,value_dict=self.str2json(sql_query)
        if sql_query is None:
            return "GPT 生成结果JSON 解码错误"
        else:
            print(sql_query)
        with open("query.json","w",encoding="utf-8") as f:
            json.dump({"query":sql_query},f,ensure_ascii=False)
        
        # 执行 SQL 查询
        ana_res=self.fetch_complex_data(self.cursor, sql_query=sql_query)
        # 提交事务
        if ana_res is None:
            return "GPT 生成的代码无法运行，哈哈哈。"
        return ana_res

    def connect_to_postgres(self):
        """
        Connects to the PostgreSQL database using global DB_CONFIG and returns the connection and cursor.
        """
        try:
            # 连接到 PostgreSQL 数据库
            conn = psycopg2.connect(**DB_CONFIG)
            print("PGSQL Connection successful")
            return conn
        except Exception as e:
            print(f"Error: {e}")
            return None
    def str2json(self,input_str:str,target='DAU_Analysis'):
        # exa_j['DAU_Analysis'][0]['SQL']['query']
        value_dict={}
        try:
            print(type(input_str))
            ana_res=json.loads(input_str)
            sql_query=ana_res[target][0]["SQL"]['query']
            # exa_j['DAU_Analysis'][0]['SQL']['query']
            if "date" in ana_res:
                value_dict["date"]=ana_res["date"]
            return sql_query,value_dict
        except Exception as e:
            print(f"Error: {e}")
            print("解码错误有问题！")
            return None,None
        
    
    def execute_query(self,cursor, query):
        """
        Executes a given SQL query using the provided cursor and returns the results.
        """
        try:
            cursor.execute(query)
            return cursor.fetchall()
        except Exception as e:
            print(f"Error: {e}")
            return None
    def fetch_complex_data(self, cursor, sql_query):
        """
        Fetches complex data based on the provided criteria.
        """
        query = sql.SQL(sql_query)
        print(sql_query)
        results = self.execute_query(cursor, query)
        if results:
            for row in results:
                print(row)
            return results
        else:
            print(f"No results found for the query.")
            return None

# agentPGSQL=AgentPGSQL()
# example=r"""{"DAU_Analysis": [{"SQL": {"query": "SELECT \n    metrics_date,\n    application_id,\n    SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) AS \"dau\"\n FROM \n    {}.{}\nWHERE metrics_date BETWEEN '2024-06-09' AND '2024-06-09'\n    AND period = 1\n    AND os_version = '#Overall#'\n    AND device_model = '#Overall#'\n    AND client_version = '#Overall#'\n    AND client_build_type = '#Overall#'\n    AND install_channel_l1 = '#Overall#'\n    AND install_channel_l2 = '#Overall#'\n    AND install_channel_l3 = '#Overall#'\n    AND install_channel_l4 = '#Overall#'\n    AND mini_app = ''\n    AND first_launch_source = '#Overall#'\n    AND launch_source = '#Overall#'\n    AND market = '#Overall#'\n    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')\nGROUP BY metrics_date, application_id"}}, "additional_metrics"], "DAU": "dau", "startdate": "2024-06-04", "enddate": "2024-06-04", "application_id_list": ["2130688B018F4B44BBED68E7A42BBA1E", "AE427635ADC245AE973038BCB3D7C21B", "4DC5714ABCAD449BA13A9B701A3CF296", "4A5B528B59954AAE8725B509A41FBF1A"], "query_conclusion": "查询指定的四个应用程序（Bing-Android、Bing-IOS、Start-Android、Start-IOS）在2024年6月4日的日活跃用户数（DAU）。", "result_count": "4", "market_analysis": {"enabled": false}}"""
# print(example[44:46])
# example=example.replace(r"{}.{}","tablename")
# print(agentPGSQL.chat(example))

In [19]:
agentSQL=AgentPGSQL()

init_agentSQL
PGSQL Connection successful
del-运行


In [33]:
market_top_10_query="""
SELECT market,
SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
FROM 
your_schema.your_table 
WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
GROUP BY market
ORDER BY dau DESC
LIMIT 10;"""

In [34]:
# market_count_query="""
# SELECT count(distinct(market))  as market_count
# FROM 
# your_schema.your_table 
# WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
# AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
# ;"""
agentSQL.chat(market_top_10_query,debug=True)

PGSQL Connection successful

SELECT market,
SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
FROM 
sapphire.sapphire_engagement_metrics_master 
WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
GROUP BY market
ORDER BY dau DESC
LIMIT 10;

SELECT market,
SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
FROM 
sapphire.sapphire_engagement_metrics_master 
WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
GROUP BY market
ORDER BY dau DESC
LIMIT 10;
('#Overall#', 705696946.0)
('en-xl', 35851980.0)
('en-us', 27332840.0)
('es-xl', 24975081.0)
('en-in', 22389759.0)
('pt-br', 15275964.0)
('zh-cn', 11140802.0)
('es-mx', 8449721.0)
('ja-jp', 8026148.0)
('de-de', 7703039.0)


[('#Overall#', 705696946.0),
 ('en-xl', 35851980.0),
 ('en-us', 27332840.0),
 ('es-xl', 24975081.0),
 ('en-in', 22389759.0),
 ('pt-br', 15275964.0),
 ('zh-cn', 11140802.0),
 ('es-mx', 8449721.0),
 ('ja-jp', 8026148.0),
 ('de-de', 7703039.0)]

In [40]:
# import pandas as pd
# # pd.set_option('display.max_colwidth', None)
# from azure.identity import AzureCliCredential, DefaultAzureCredential
# from openai import AzureOpenAI
# import os
# from openai import AzureOpenAI
# from azure.identity import DefaultAzureCredential, get_bearer_token_provider
# import json
# #  system_prompt
# """

# # Step 2: 导入所需模块
# from azure.identity import AzureCliCredential, DefaultAzureCredential
# from openai import AzureOpenAI

# # Step 3: 获取 Azure CLI 凭据
# credential = AzureCliCredential()

# """
# # export PATH=$PATH:/home/jason00

# dau_analysis_sql_query=""" SELECT 
# metrics_date, 
# application_id, 
# SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
# FROM 
# your_schema.your_table 
# WHERE metrics_date BETWEEN '2024-06-09' AND '2024-06-09' AND period = 1    
# AND os_version = '#Overall#'     
# AND device_model = '#Overall#'    
# AND client_version = '#Overall#' 
# AND client_build_type = '#Overall#'    
# AND install_channel_l1 = '#Overall#'    
# AND install_channel_l2 = '#Overall#'    
# AND install_channel_l3 = '#Overall#'    
# AND install_channel_l4 = '#Overall#'    
# AND mini_app = ''     
# AND first_launch_source = '#Overall#'    
# AND launch_source = '#Overall#' 
# AND market = '#Overall#'     
# AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B') 
# GROUP BY metrics_date, application_id"""

# wau_analysis_sql_query="""SELECT\n    metrics_date AS date,\n    application_id,\n    SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) AS wau\nFROM\n    your_schema.your_table\n WHERE\n    metrics_date BETWEEN '2024-06-09' AND '2024-06-03'\n    AND period = 7\n    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')\nGROUP BY\n    metrics_date, application_id;"""
# # # mau_analysis_sql_query="SELECT\n    metrics_date AS date,\n    application_id,\n    SUM(CAST(metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION)) AS mau\nFROM\n    your_schema.your_table\n WHERE\n    metrics_date BETWEEN '2024-06-09' AND '2024-05-09'\n    AND period = 30\n    AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')\nGROUP BY\n    metrics_date, application_id;"
# market_top_10_dau_query="""
# SELECT market,
# SUM( CAST( metrics->'ActiveUser'->'uu'->>'uu' AS DOUBLE PRECISION ) ) AS dau 
# FROM 
# your_schema.your_table 
# WHERE metrics_date BETWEEN  '2024-06-09' AND '2024-06-09'
# AND application_id IN ('2130688B018F4B44BBED68E7A42BBA1E', 'AE427635ADC245AE973038BCB3D7C21B')
# GROUP BY market
# ORDER BY dau DESC
# LIMIT 10;"""
# json_fewshot=[
#   {
#     "Analysis": {
#         "input":"Bing APP一天有多少用户？",
#         "description":"获取在2024年6月9日这一天，指定的两个应用程序（Bing-Android和Bing-IOS）的日活跃用户数（DAU）",
#         "SQL": dau_analysis_sql_query,
#     },
#   },
#   {
#     "Analysis": {
#         "input":"Bing APP一周有多少用户？",
#         "description":"获取在2024年6月3日至6月9日这一周，指定的两个应用程序（Bing-Android和Bing-IOS）的日活跃用户数（DAU）",
#         "SQL": wau_analysis_sql_query,
#     },
#   },
#   {
#     "Analysis": {
#         "input":"告诉我Bing APP日活最多的一个前十个市场",
#         "description":"获取在2024年6月9日这一天，指定的两个应用程序（Bing-Android和Bing-IOS）的日活跃用户数（DAU）,先对其进行降序排序，然后选取前10个dau最大的项。",
#         "SQL": market_top_10_dau_query,
#     },
#   },
# ]


# prompt_dau = """
# Context: 
# Now we have the user data for some apps, these app are search apps, it intagrated the copilot for AI.
# User can also search for information, this app get revenue by ads, and it also have many miniapps for different feature.
# We did some AB test based on user interaction data from an app, 
# which includes various metrics such as user behavior, codex (AI) behavior, and search behavior. 

# The data for each user is structured as a sentence.

# The goal is to generate SQL and more information.
# The query should focus on：
# market:
# Format is "en-us",first two is language, last two is country(xl means other counrty that we can not get info.)
# Eg."en-us" means American people speak English.

# application_id: user will describe a app name, and you need to use id when you serach.
# WHEN application_id = '2130688B018F4B44BBED68E7A42BBA1E' THEN 'Bing-Android'
# application_id = 'AE427635ADC245AE973038BCB3D7C21B' THEN 'Bing-IOS'
# application_id = '4DC5714ABCAD449BA13A9B701A3CF296' THEN 'Start-Android'
# application_id = '4A5B528B59954AAE8725B509A41FBF1A' THEN 'Start-IOS' 
# application_id = 'F185A93DE6764B098D55089F610A3FB8' THEN 'Copilot-Android'
# application_id = 'FC320C411FC12CD4DFBE9A00F3161364' THEN 'Copilot-IOS'
# If user did not give you os name, you need to Get Both of them.
# Eg."Bing"means ['2130688B018F4B44BBED68E7A42BBA1E','AE427635ADC245AE973038BCB3D7C21B']

# period: 1 for DAU, 7 for WOW, and 30 for MOM

# Here is the schema of table:                                                                                                                                                                                                                                                                
# umid                 | 1736681
# metrics_date         | 2022-09-23
# period               | 1
# application_id       | 4DC5714ABCAD449BA13A9B701A3CF296
# market               | en-in
# os_version           | #Overall#
# device_model         | #Overall#
# client_version       | 23.3.400914606
# client_build_type    | #Overall#
# install_channel_l1   | #Overall#
# install_channel_l2   | #Overall#
# install_channel_l3   | #Overall#
# install_channel_l4   | #Overall#
# mini_app             | 
# first_launch_source  | #Overall#
# launch_source        | #Overall#
# metrics              | {'ActiveUser': {'uu': {'uu': 467.0}, 'msa': {'msa_uu': 0.0}, 'msb': {'msb_uu': 0.0}, 'session': {'session_cnt': 894.0}, 'dwelltime': {'dwelltime': 107686.0, 'dwelltime_uu': 382.0}}}


# Output Format: 
# Each set of data returns one json, followed by an example based on the provided data.

# result_count: How many result will get after query, for example, if group by 2 date and 3 application, it will get 2*3=6 columns.
# if you guess the column will more than 100, eg. group by market, please use 100.

# Specifications:
# - Analyze the input and describe their demand.
# - Use these column to identify and describe user demand.
# - Provide a detailed and correct pgsql query for each input.
# - Limit the output length to avoid excessive verbosity.

# Task Examples: 


# """
# # json_fewshot_str=json.dumps(json_fewshot,ensure_ascii=False)
# for idx,shot in enumerate(json_fewshot):
#     prompt_dau+="Input"+str(idx)+"\n"+shot["Analysis"]["input"]+":\n"
#     prompt_dau+="Output"+str(idx)+"\n"+json.dumps(shot,ensure_ascii=False)+":\n"
# print(prompt_dau)


Context: 
Now we have the user data for some apps, these app are search apps, it intagrated the copilot for AI.
User can also search for information, this app get revenue by ads, and it also have many miniapps for different feature.
We did some AB test based on user interaction data from an app, 
which includes various metrics such as user behavior, codex (AI) behavior, and search behavior. 

The data for each user is structured as a sentence.

The goal is to generate SQL and more information.
The query should focus on：
market:
Format is "en-us",first two is language, last two is country(xl means other counrty that we can not get info.)
Eg."en-us" means American people speak English.

application_id: user will describe a app name, and you need to use id when you serach.
WHEN application_id = '2130688B018F4B44BBED68E7A42BBA1E' THEN 'Bing-Android'
application_id = 'AE427635ADC245AE973038BCB3D7C21B' THEN 'Bing-IOS'
application_id = '4DC5714ABCAD449BA13A9B701A3CF296' THEN 'Start-Android'
